<a href="https://colab.research.google.com/github/drcjaudet/GT_Thyroide_publique/blob/main/AnalyseGTthyroide_statistique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#importation des librairies

In [ ]:
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
from scipy import stats
from scipy.stats import pearsonr
from scipy.stats import ttest_ind
import pandas as pd
import numpy as np
from numpy.core.numeric import NaN
import tensorflow as tf
from tensorflow import keras
import random
import sklearn
from keras.callbacks import ModelCheckpoint
import os
import seaborn as sn
import math


from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

%pip install keras-tuner --upgrade
import keras_tuner as kt
from keras import regularizers
########


!pip install scikit-posthocs
import pandas as pad
import statistics
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import numpy as np
from scipy.stats import pearsonr
from scipy.stats import ttest_ind


from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import scikit_posthocs as sp

import statsmodels.api as sm
from statsmodels.formula.api import ols

import statsmodels.stats.multicomp as multi



#Importation des données

In [ ]:
#importation des données
df_data = pd.read_csv('/content/data_GTthyroide/Recueil_global.csv',sep=";",encoding='latin1') #
df_data

In [ ]:
type(df_data)

In [ ]:
#remplace , par .
df_data = df_data.replace(',','.', regex=True)

In [ ]:
#Supprimer les lignes avec des Nan: (Pas bonne idée ;)
#index_with_nan = df_data.index[df_data.isnull().any(axis=1)]
#df_data.drop(index_with_nan,0, inplace=True)

#df_data

In [ ]:
df_data.head()

In [ ]:
df_data.columns

In [ ]:
df_data["sensibilite (Cps/(MBq.s))_5%"][0]

#prétraitement des données

In [ ]:
list_sensibilite=["sensibilite (Cps/(MBq.s))_5%","sensibilite (Cps/(MBq.s))_10%","sensibilite (Cps/(MBq.s))_15%","sensibilite (Cps/(MBq.s))_20%","sensibilite (Cps/(MBq.s))_25%","sensibilite (Cps/(MBq.s))_30%","sensibilite (Cps/(MBq.s))_35%","sensibilite (Cps/(MBq.s))_40%"]
List_ErreurRelative=['sensibiliteError_5%','sensibiliteError_10%','sensibiliteError_15%','sensibiliteError_20%','sensibiliteError_25%','sensibiliteError_30%','sensibiliteError_35%','sensibiliteError_40%']

In [ ]:
df_data["fant"].unique()

In [ ]:
#Création d'une collone d'erreur relative sur la mesure d'activité par rapport à seringue seuil 10%

liste_centre_selection=df_data["Centre"].unique()

df_data['sensibiliteError_5%'] = ''
df_data['sensibiliteError_10%'] = ''
df_data['sensibiliteError_15%'] = ''
df_data['sensibiliteError_20%'] = ''
df_data['sensibiliteError_25%'] = ''
df_data['sensibiliteError_30%'] = ''
df_data['sensibiliteError_35%'] = ''
df_data['sensibiliteError_40%'] = ''



In [ ]:
# Calculating the relative error on sensitivity
for centre in liste_centre_selection:
    Data_Centre = df_data[df_data["Centre"] == centre]

    if not Data_Centre.empty:
        liste_config = Data_Centre["config"].unique()

        for config in liste_config:
            Data_config = Data_Centre[Data_Centre["config"] == config]

            if not Data_config.empty:
                liste_protocole = Data_config["Proto"].unique()

                for protocole in liste_protocole:
                    Data_protocole = Data_config[Data_config["Proto"] == protocole]

                    if not Data_protocole.empty:
                        # Determine normalization criteria based on the protocol
                        NormBy ='IRSN-15'  if protocole == 'Std' else 'Routine' # 'IRSN-15'

                        # Get the normalization value
                        ValeurNorm = Data_protocole[Data_protocole["fant"] == NormBy]

                        if not ValeurNorm.empty:
                            ValeurNorm = float(ValeurNorm["sensibilite (Cps/(MBq.s))_10%"].iloc[0])
                            liste_fantome = Data_protocole["fant"].unique()

                            for fantome in liste_fantome:
                                Data_fantome = Data_protocole[Data_protocole["fant"] == fantome]

                                if not Data_fantome.empty and ValeurNorm != 0:
                                    for i in range(len(list_sensibilite)):
                                        try:
                                            # Calculate the relative error
                                            Y = ((float(Data_fantome[list_sensibilite[i]].iloc[0]) - ValeurNorm) / ValeurNorm) * 100

                                            # Update the DataFrame in one step using .loc[] to avoid chained assignment
                                            df_data.loc[
                                                (df_data["Centre"] == centre) &
                                                (df_data["config"] == config) &
                                                (df_data["Proto"] == protocole) &
                                                (df_data["fant"] == fantome),
                                                List_ErreurRelative[i]
                                            ] = Y
                                        except Exception as e:
                                            print(f"An error occurred: {e}")


In [ ]:
# storing in new variable
data_top = df_data.head()

# iterating the columns
for row in data_top:
    print(row, end = " ")

In [ ]:
df_data["modele_colli"].unique()

In [ ]:
df_data["Collimateur"].unique()

In [ ]:
df_data["fant"].unique()

In [ ]:
df_data["Centre"].unique()

In [ ]:
df_data['Proto'].unique()

In [ ]:
df_data["Radionucleide_simple"].unique()

In [ ]:
df_data["NaI_CZT"].unique()

In [ ]:
df_data["Pixel"]

#selection des données

In [ ]:
#si pas de sélection:
df_data_TrierCollimatteur=df_data

###selectionne les numeriques:
df_data_TrierCollimatteur =df_data_TrierCollimatteur.loc[df_data_TrierCollimatteur["NaI_CZT"] == 'NaI'] #enléve les CZT

###selectionne les fantomes bien remplient:
#df_data_TrierCollimatteur =df_data_TrierCollimatteur.loc[df_data_TrierCollimatteur["limiteCDM2STD"] == 1] #enléve les fantome donc le cdm bouge de plus de 2*STD

Collimateur='Pinhole' #enléve les Pinhole "Pinhole" ou PARA
Proto='Local' # Std ou Local
Radionucleide_simple='I' #ou I, Tc

name_serie='data_'+Collimateur+'_'+Proto+'_'+Radionucleide_simple

###selectionne les collimateurs //:
df_data_TrierCollimatteur =df_data_TrierCollimatteur.loc[df_data_TrierCollimatteur["Collimateur"] == Collimateur] #selectionne les pinholes "Pinhole" ou PARA
###enléve les fantome de routine:
df_data_TrierCollimatteur = df_data_TrierCollimatteur.loc[df_data_TrierCollimatteur['fant'] != 'Routine']
###enléve les fantome de calibration seringue:
df_data_TrierCollimatteur = df_data_TrierCollimatteur.loc[df_data_TrierCollimatteur['fant'] != 'Seringue']
#selectionne protocole
df_data_TrierCollimatteur = df_data_TrierCollimatteur.loc[df_data_TrierCollimatteur['Proto'] ==Proto]  # Std ou Local
#selectionne iode ou Tc
df_data_TrierCollimatteur = df_data_TrierCollimatteur.loc[df_data_TrierCollimatteur["Radionucleide_simple"] == Radionucleide_simple] #ou I, Tc
#selectionne iode ou Tc
#df_data_TrierCollimatteur = df_data_TrierCollimatteur.loc[df_data_TrierCollimatteur["Marque"] == 'GE'] #ou I
#selectionne collimatteur
#df_data_TrierCollimatteur = df_data_TrierCollimatteur.loc[df_data_TrierCollimatteur['modele_colli'] != 'lHR'] #ou I #enléve les czt

#df_data_TrierCollimatteur[["Collimateur","fant","Proto"]]
#print(df_data_TrierCollimatteur.count().)



In [ ]:
list_seuil=["_5%","_10%","_15%","_20%","_25%","_30%","_35%","_40%"]
list_seuil_max=[" Seuil(% max)_5%"," Seuil(% max)_10%"," Seuil(% max)_15%"," Seuil(% max)_20%"," Seuil(% max)_25%"," Seuil(% max)_30%"," Seuil(% max)_35%"," Seuil(% max)_40%"]#"Seuil(% max)_40%"]
list_sensibilite=["sensibilite (Cps/(MBq.s))_5%","sensibilite (Cps/(MBq.s))_10%","sensibilite (Cps/(MBq.s))_15%","sensibilite (Cps/(MBq.s))_20%","sensibilite (Cps/(MBq.s))_25%","sensibilite (Cps/(MBq.s))_30%","sensibilite (Cps/(MBq.s))_35%","sensibilite (Cps/(MBq.s))_40%"]
List_ErreurRelative=['sensibiliteError_5%','sensibiliteError_10%','sensibiliteError_15%','sensibiliteError_20%','sensibiliteError_25%','sensibiliteError_30%','sensibiliteError_35%','sensibiliteError_40%']
list_sensibilite

In [ ]:
###valeur à zéros?
df_data_TrierCollimatteur
#export to test:
#df_data_TrierCollimatteur.to_csv('/content/cycy/donneepretraiter.csv')
#que faut-il garder:
#"Marque","Modele_simple", "modele_colli", "Taille_Cristal", "Pixel", "Volume", "Distance", "A_acq_corr", "Duree", "Facteur_calibrage"

In [ ]:
# storing in new variable
data_top = df_data_TrierCollimatteur.head()

# iterating the columns
for row in data_top:
    print(row, end = " ")

In [ ]:
#df_data_TrierCollimatteur[' Seuil(% max)_5%']

In [ ]:
#reformat 1 ligne par seuil:,"CDM normalise"
Df_trier = pd.DataFrame()
for i in range(len(list_seuil_max)):
  #print(i) "Marque", "modele_colli",
  #list_extraction=["Modele","Marque","Pixel","Volume",list_seuil_max[i],list_sensibilite[i],List_ErreurRelative[i]]#"modele_colli","modele_colli","Taille_Cristal",
  #list_extraction=["Modele","Marque","modele_colli","Taille_Cristal","Pixel","Volume",list_seuil_max[i],list_sensibilite[i],List_ErreurRelative[i]]#"modele_colli",
  #list_extraction=["Centre","config","Modele","Marque","modele_colli","Taille_Cristal","Pixel","Duree","Distance","Volume","Radionucleide_simple",list_seuil_max[i],list_sensibilite[i],List_ErreurRelative[i]]#
  #list_extraction=["Marque","Modele","modele_colli","Taille_Cristal","Pixel","Duree","Distance","Volume",list_seuil_max[i],list_sensibilite[i],List_ErreurRelative[i]]#
  #list_extraction=["Centre","Marque","Modele_simple", "modele_colli", "Taille_Cristal", "Pixel", "Volume", "Distance", "Duree", "A_acq_corr" ,list_seuil_max[i],list_sensibilite[i],List_ErreurRelative[i]]# pour colli // local
  list_extraction=["Centre","Marque","Modele_simple","facteur_stenope", "Taille_Cristal", "Pixel", "Volume", "Distance", "Duree","A_acq_corr", list_seuil_max[i],list_sensibilite[i],List_ErreurRelative[i]]# pour colli sténopé local
  #list_extraction=["Centre","Marque","Modele_simple", "modele_colli", "Taille_Cristal", "Pixel", "Volume", "A_acq_corr" ,list_seuil_max[i],list_sensibilite[i],List_ErreurRelative[i]]# pour colli // std
  #list_extraction=["Centre","Marque","Modele_simple","Taille_Cristal", "facteur_stenope", "Pixel", "Volume", "A_acq_corr" ,list_seuil_max[i],list_sensibilite[i],List_ErreurRelative[i]]# pour colli sténopé std , "facteur_stenope",
  Df_temporary=df_data_TrierCollimatteur[list_extraction].copy()
  Df_temporary=Df_temporary.rename(columns={list_seuil_max[i]: "Seuil", list_sensibilite[i]: "Sensibilite",List_ErreurRelative[i]:"ErreurRelative"})
  Df_trier=pd.concat([Df_trier,Df_temporary], axis=0)

Df_trier

In [ ]:
##vire les nan
Df_trier=Df_trier.dropna(axis=0)

In [ ]:
#Elimine les zeros et valeur bizzare qui pourrait rester?
Df_trier['Sensibilite'].unique()

In [ ]:
#Df_trier["modele_colli"].unique()

In [ ]:
#Protocole standardisé:
#Ne garde qu'un seuil:
###selectionne seuil
#Df_trier = Df_trier.loc[Df_trier["Seuil"] == 10] #ou I
#Df_trier=Df_trier.drop(columns=['Seuil'])
###Ne garde qu'une durée:
#Df_trier = Df_trier.loc[Df_trier["Duree"] == 300.0] #ou I
#Df_trier=Df_trier.drop(columns=['Duree'])
###Ne garde qu'une distance:
#Df_trier = Df_trier.loc[Df_trier["Distance"] == 8.0] #ou I
#Df_trier=Df_trier.drop(columns=['Distance'])

In [ ]:
Df_trier

In [ ]:
#convertion de type:

#Df_trier["Facteur_calibrage"]=Df_trier["Facteur_calibrage"].fillna(0).astype('float64')
#Df_trier["A_acq_corr"]=Df_trier["A_acq_corr"].fillna(0).astype('float64')
Df_trier["Seuil"]=Df_trier["Seuil"].fillna(0).astype('float64')
Df_trier["Distance"]=Df_trier["Distance"].fillna(0).astype('float64')
Df_trier["Volume"]=Df_trier["Volume"].fillna(0).astype('float64')
Df_trier["Sensibilite"]=pd.to_numeric(Df_trier["Sensibilite"].fillna(0))#.astype('float64')
Df_trier["ErreurRelative"]=pd.to_numeric(Df_trier["ErreurRelative"].fillna(0))#.astype('float64')
Df_trier["Taille_Cristal"]=pd.to_numeric(Df_trier["Taille_Cristal"].fillna(0))#.astype('float64')
Df_trier["Pixel"]=pd.to_numeric(Df_trier["Pixel"].fillna(0))#.astype('float64')
Df_trier["Duree"]=pd.to_numeric(Df_trier["Duree"].fillna(0))#.astype('float64')
Df_trier['Centre'] = pd.to_numeric(Df_trier['Centre'].fillna(0))
#Df_trier['facteur_stenope'] = pd.to_numeric(Df_trier['facteur_stenope'].fillna(0))
#Df_trier['config'] = pd.to_numeric(Df_trier['config'].fillna(0))
Df_trier

In [ ]:
#categoriel to numerique:
# Importing LabelEncoder from Sklearn
# library from preprocessing Module.
from sklearn.preprocessing import LabelEncoder

# Creating a instance of label Encoder.
le = LabelEncoder()

# Using .fit_transform function to fit label
# encoder and return encoded label



Df_trier['Modele_simple'] = le.fit_transform(Df_trier['Modele_simple'])
#Df_trier["Radionucleide_simple"] = le.fit_transform(Df_trier["Radionucleide_simple"])
#Df_trier['Centre'] = le.fit_transform(Df_trier['Centre'])
#Df_trier['config'] = le.fit_transform(Df_trier['config'])
#Df_trier['Proto'] = le.fit_transform(Df_trier['Proto'])
Df_trier['Marque'] = le.fit_transform(Df_trier['Marque'])
#Df_trier['modele_colli'] = le.fit_transform(Df_trier['modele_colli'])
#Df_trier['Radionucleide_simple'] = le.fit_transform(Df_trier['Radionucleide_simple'])
# printing label
Df_trier


In [ ]:
#Df_trier[Df_trier['Distance']!=8.0]

In [ ]:
#Choix d'un seuil:
#Df_trier[Df_trier['Seuil']==10.0]
#élimination de la colone
#Df_trier=Df_trier.drop(columns=['Seuil'])

In [ ]:
#Choix d'un protocole:
#Df_trier[Df_trier['Proto']==0]
#élimination de la colone
#Df_trier=Df_trier.drop(columns=['Proto'])

In [ ]:
#Df_trier["Centre"]

In [ ]:
Df_trier.describe().T

#bilan sur les données:

In [ ]:
Df_trier.describe().T

In [ ]:
total = Df_trier.isnull().sum().sort_values(ascending=False)
percent = (Df_trier.isnull().sum()/Df_trier.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data[missing_data['Total'] > 0]

In [ ]:
#elimine si trop de données manquante:
Df_trier = Df_trier[missing_data[missing_data['Percent'] < 0.15].index]

In [ ]:
#distribution des données:
Df_trier.hist(bins=50, figsize=(30,20));

In [ ]:
####analyse visuelle des correlations

#sns.set()
#cols =["Centre","config","fant","Marque","Modele","modele_colli","Taille_Cristal","Pixel","Radionucleide_simple","Proto","Duree","Distance","Volume","taille Matrice X","CDM normalise"]#,list_seuil_max[i],list_sensibilite[i],List_ErreurRelative[i]]
##cols =Df_trier.columns
#sns.pairplot(Df_trier, height = 2.5)
#plt.show();

In [ ]:
Df_trier.columns

#graph finalisé pour le rapport:

In [ ]:
Df_trier.describe().T

In [ ]:
#boxplot test sensibilité:
var = 'Centre'
data = pd.concat([Df_trier['Sensibilite'], Df_trier[var]], axis=1)
f, ax = plt.subplots(figsize=(16, 8))
fig = sns.boxplot(x=var, y="Sensibilite", data=data)
fig.set_ylim([0, 100])
plt.xticks(rotation=90);
plt.title(name_serie)#"Data_local_Pinhole_Tc99m_PasCZT "

In [ ]:
#boxplot test Erreur relative:
var = 'Centre'
data = pd.concat([Df_trier['ErreurRelative'], Df_trier[var]], axis=1)
f, ax = plt.subplots(figsize=(16, 8))
fig = sns.boxplot(x=var, y="ErreurRelative", data=data)
fig.set_ylim([-75, 75])
plt.xticks(rotation=90);
plt.title(name_serie)

In [ ]:
###virer le centre ;)
Df_trier=Df_trier.drop(['Centre'], axis=1)
Df_trier=Df_trier.drop(['A_acq_corr'], axis=1)


In [ ]:
#test de distribution normal:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
donnee=Df_trier['Sensibilite']
#test de Shapiro–Wilk
#The Shapiro-Wilk test tests the null hypothesis that the data was drawn from a normal distribution.
from scipy.stats import shapiro
print(shapiro(donnee))
#test d'Anderson-Darling permet de tester d'autre distribution:
from scipy.stats import anderson
print(anderson(donnee, dist='norm'))
#test D’Agostino (normaltest)
from scipy.stats.mstats import normaltest
print(normaltest(donnee, axis=0))
#test de Kolmogorov-Smirnov?
sns.distplot(Df_trier['Sensibilite'], fit = norm)

In [ ]:
import statsmodels
from scipy.stats import kendalltau, pearsonr, spearmanr

def kendall_pval(x,y):
  return kendalltau(x,y)[1]

def pearsonr_pval(x,y):
  return pearsonr(x,y)[1]

def spearmanr_pval(x,y):
  return spearmanr(x,y)[1]

In [ ]:
# 'Sensibilite' correlation matrix
#df_marks = df_marks.drop(['chemistry'], axis=1) #enléve l'erreur
k = 13
 #number of variables for heatmap
corrmat = Df_trier.corr(method='spearman')
ix = corrmat.sort_values('Sensibilite', ascending=False).index
Df_trier_sorted = Df_trier.loc[:, ix]
corrmat_sorted = Df_trier_sorted.corr(method='spearman')
f, ax = plt.subplots(figsize=(12, 10))
cols=corrmat_sorted.index
#plt.xticks(rotation=0)
#plt.yticks(rotation=0)
ax.tick_params(axis='x', labelrotation=0)
ax.tick_params(axis='y', labelrotation=90)
sns.heatmap(corrmat_sorted, ax=ax, cmap="coolwarm", linewidths=0.3, yticklabels=cols.values, xticklabels=cols.values,annot=True,vmax=1, vmin=-1, center=0,  annot_kws={'size': 16}, square=True)

#si on veut rajouter le p values:
#https://tosinharold.medium.com/enhancing-correlation-matrix-heatmap-plots-with-p-values-in-python-41bac6a7fd77
#p_values =  Df_trier_sorted.corr(method=spearmanr_pval)
#mask_p = np.invert(np.tril(p_values<0.05))
#sns.heatmap(corrmat_sorted, ax=ax, cmap="coolwarm", linewidths=0.1, yticklabels=cols.values, xticklabels=cols.values,annot=True,vmax=1, vmin=-1, center=0, mask=mask_p)




In [ ]:
#sensibilités names, coor, pvalue, pavlue corrected:
name='Sensibilite' #valeur avec lesquelle on recherche une correlation
corr_pvalue =  Df_trier.corr(method=spearmanr_pval)
#Sens_corr=[[]]
Df_Corr_Sens= pd.DataFrame()
Df_Corr_Sens['corr']=corrmat[name]
Df_Corr_Sens['Abs(corr)']=abs(corrmat[name])
Df_Corr_Sens['pvalue']=corr_pvalue[name].values
Df_Corr_Sens=Df_Corr_Sens.drop('Sensibilite', axis=0)
Df_Corr_Sens=Df_Corr_Sens.drop('ErreurRelative', axis=0)
Df_Corr_Sens=Df_Corr_Sens.sort_values(by='Abs(corr)', ascending=False)
#corr_pvalue['Sensibilite'].values
Df_Corr_Sens['benjaminiHochberg']=statsmodels.stats.multitest.multipletests(Df_Corr_Sens['pvalue'].values, alpha=0.05, method='fdr_bh', is_sorted=True, returnsorted=False)[0] #fdr_bh : Benjamini/Hochberg (non-negative)

Df_Corr_Sens

In [ ]:
Df_Corr_Sens['corr']

In [ ]:
#test de distribution normal:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
donnee=Df_trier['ErreurRelative']
#test de Shapiro–Wilk
#The Shapiro-Wilk test tests the null hypothesis that the data was drawn from a normal distribution.
from scipy.stats import shapiro
print(shapiro(donnee))
#test d'Anderson-Darling permet de tester d'autre distribution:
from scipy.stats import anderson
print(anderson(donnee, dist='norm'))
#test D’Agostino (normaltest)
from scipy.stats.mstats import normaltest
print(normaltest(donnee, axis=0))
#test de Kolmogorov-Smirnov?
sns.distplot(Df_trier['ErreurRelative'], fit = norm)

In [ ]:
# 'Sensibilite' correlation matrix
#df_marks = df_marks.drop(['chemistry'], axis=1) #enléve l'erreur
k = 13
 #number of variables for heatmap
corrmat = Df_trier.corr(method='spearman')
ix = corrmat.sort_values('ErreurRelative', ascending=False).index
Df_trier_sorted = Df_trier.loc[:, ix]
corrmat_sorted = Df_trier_sorted.corr(method='spearman')
f, ax = plt.subplots(figsize=(12, 10))
cols=corrmat_sorted.index
sns.heatmap(corrmat_sorted, ax=ax, cmap="coolwarm", linewidths=0.3, yticklabels=cols.values, xticklabels=cols.values,annot=True,vmax=1, vmin=-1, center=0,  annot_kws={'size': 16}, square=True)

#si on veut rajouter le p values:
#https://tosinharold.medium.com/enhancing-correlation-matrix-heatmap-plots-with-p-values-in-python-41bac6a7fd77

In [ ]:
#'ErreurRelative' names, coor, pvalue, pavlue corrected:
name='ErreurRelative' #valeur avec lesquelle on recherche une correlation
corr_pvalue =  Df_trier.corr(method=spearmanr_pval)
#Sens_corr=[[]]
Df_Corr_ER= pd.DataFrame()
Df_Corr_ER['corr']=corrmat[name]
Df_Corr_ER['Abs(corr)']=abs(corrmat[name])
Df_Corr_ER['pvalue']=corr_pvalue[name].values
Df_Corr_ER=Df_Corr_ER.drop('Sensibilite', axis=0)
Df_Corr_ER=Df_Corr_ER.drop('ErreurRelative', axis=0)
Df_Corr_ER=Df_Corr_ER.sort_values(by='Abs(corr)', ascending=False)
#corr_pvalue['Sensibilite'].values
Df_Corr_ER['benjaminiHochberg']=statsmodels.stats.multitest.multipletests(Df_Corr_ER['pvalue'].values, alpha=0.05, method='fdr_bh', is_sorted=True, returnsorted=False)[0] #fdr_bh : Benjamini/Hochberg (non-negative)

Df_Corr_ER

#redaction rapport